In [488]:
import pandas as pd
import pathlib
import shutil

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

DATA_DIR = 'исходные данные'

OUT_DIR = 'processed'
def to_csv(df: pd.DataFrame, name: str):
    df.to_csv(f'{OUT_DIR}/{name}.csv', index=False)

DATA_FILENAME = 'MOCK_DATA'
df = pd.read_csv(f'{DATA_DIR}/{DATA_FILENAME}.csv')

for i in range(1, 10):
    t = pd.read_csv(f'{DATA_DIR}/{DATA_FILENAME} ({i}).csv')
    t[[c for c in t.columns if 'id' in c.lower()]] += 1000 * i
    t.index = t.index + 1000 * i
    df = pd.concat([df, t])

shutil.rmtree(pathlib.Path(OUT_DIR))
pathlib.Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
to_csv(df, 'whole_mock_data')

df

,id,customer_first_name,customer_last_name,customer_age,customer_email,customer_country,customer_postal_code,customer_pet_type,customer_pet_name,customer_pet_breed,seller_first_name,seller_last_name,seller_email,seller_country,seller_postal_code,product_name,product_category,product_price,product_quantity,sale_date,sale_customer_id,sale_seller_id,sale_product_id,sale_quantity,sale_total_price,store_name,store_location,store_city,store_state,store_country,store_phone,store_email,pet_category,product_weight,product_color,product_size,product_brand,product_material,product_description,product_rating,product_reviews,product_release_date,product_expiry_date,supplier_name,supplier_contact,supplier_email,supplier_phone,supplier_address,supplier_city,supplier_country
0,1,Barron,Rawlyns,61,bmassingham0@army.mil,China,NaN,cat,Priscella,Labrador Retriever,Bevan,Massingham,bmassingham0@answers.com,Indonesia,NaN,Dog Food,Food,77.97,89,5/14/2021,1,1,1,4,487.70,Youopia,Suite 75,Xichehe,NaN,United States,564-244-8660,bmassingham0@networkadvertising.org,Cats,13.4,Indigo,Medium,Skajo,Steel,Aliquam quis turpis eget elit sodales sceleris...,2.1,97,10/19/2011,10/21/2028,Tagcat,Bevan Massingham,bmassingham0@unblog.fr,914-877-7062,Suite 25,Kletek,China
1,2,Ham,Knowller,78,cscudder1@time.com,Poland,73-115,bird,Dalenna,Labrador Retriever,Candide,Scudder,cscudder1@xing.com,Guatemala,04015,Cat Toy,Food,48.70,73,11/13/2021,2,2,2,10,484.61,Fivespan,Suite 1,Dolice,NaN,Lithuania,596-788-0321,cscudder1@behance.net,Cats,36.3,Khaki,Small,Wordpedia,Plexiglass,Phasellus in felis. Donec semper sapien a libe...,2.4,37,4/17/2019,2/29/2028,Livetube,Candide Scudder,cscudder1@sbwire.com,863-319-5653,18th Floor,Santa Cruz Balanyá,Poland
2,3,Farleigh,Langley,71,vhuxter2@fotki.com,Poland,56-160,bird,Aldridge,Parakeet,Vinson,Huxter,vhuxter2@issuu.com,China,NaN,Bird Cage,Food,77.75,39,12/4/2021,3,3,3,9,144.24,Avamba,Suite 69,Wińsko,NaN,South Africa,148-671-4697,vhuxter2@marketwatch.com,Fish,14.6,Khaki,Medium,Riffpedia,Plexiglass,"Maecenas leo odio, condimentum id, luctus nec,...",3.0,218,2/3/2010,9/3/2023,Photobug,Vinson Huxter,vhuxter2@slate.com,434-817-1275,Apt 96,Huangpu,Samoa
3,4,Shae,Debling,28,bbullier3@bravesites.com,Sri Lanka,81700,cat,Beverie,Labrador Retriever,Bald,Bullier,bbullier3@imageshack.us,China,NaN,Dog Food,Food,54.18,68,8/10/2021,4,4,4,4,441.09,Feedfish,Suite 77,Weligama,NaN,Cuba,190-352-7524,bbullier3@infoseek.co.jp,Fish,15.7,Indigo,Medium,Flipstorm,Granite,Nullam sit amet turpis elementum ligula vehicu...,2.2,959,8/22/2019,3/29/2026,Janyx,Bald Bullier,bbullier3@dot.gov,515-756-7392,12th Floor,Chengzhong,Indonesia
4,5,Lay,Twatt,25,sedgar4@smugmug.com,Botswana,NaN,bird,Sydelle,Siamese,Sydelle,Edgar,sedgar4@sogou.com,China,NaN,Bird Cage,Cage,72.96,23,2/4/2021,5,5,5,6,72.27,Skimia,Apt 112,Mahalapye,NaN,Philippines,364-385-6169,sedgar4@e-recht24.de,Reptiles,38.9,Maroon,Large,Zazio,Aluminum,Quisque id justo sit amet sapien dignissim ves...,4.5,130,12/9/2018,5/10/2030,Lazz,Sydelle Edgar,sedgar4@reverbnation.com,861-980-8811,Suite 79,Lanxi,China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Perry,Edwins,49,mkardosrn@jimdo.com,Indonesia,NaN,cat,Warden,Labrador Retriever,Milty,Kardos,mkardosrn@miitbeian.gov.cn,Brazil,09500-000,Dog Food,Toy,17.02,53,4/7/2021,9996,9996,9996,7,214.00,Wikivu,7th Floor,Sidomukti,NaN,Venezuela,404-942-8512,mkardosrn@hp.com,Dogs,2.9,Maroon,Medium,Twimbo,Plexiglass,Phasellus sit amet erat. Nulla tempus. Vivamus...,2.6,355,1/15/2019,4/9/2024,Quire,Milty Kardos,mkardosrn@barnesandnoble.com,688-242-0789,Suite 47,São Caetano do Sul,Brazil
9996,9997,Benedick,Sherwell,71,ikingzettro@salon.com,El Salvador,NaN,bird,Lezley,Labrador Retriever,Isidora,Kingzett,ikingzettro@blogtalkradio.com,China,NaN,Bird Cage,Food,57.64,81,2/22/2021,9997,9997,9997,4,495.11,Mymm,Apt 1734,Guazapa,NaN,

In [489]:
country_cols = [c for c in df.columns if 'country' in c.lower()]
countries = pd.DataFrame({
    'country': pd.concat([df[c] for c in country_cols]).dropna().unique()
})
countries['country_id'] = countries.index + 1
to_csv(countries, 'countries')
print(f'PK is country:\n{countries.nunique()}')
print(countries)

city_cols = [c for c in df.columns if 'city' in c.lower()]
cities = pd.DataFrame({
    'city': pd.concat([df[c] for c in city_cols]).dropna().unique()
})
cities['city_id'] = cities.index + 1
to_csv(cities, 'cities')
print(f'\nPK is city:\n{cities.nunique()}')
print(cities)

PK is country:
country       230
country_id    230
dtype: int64
                         country  country_id
0                          China           1
1                         Poland           2
2                      Sri Lanka           3
3                       Botswana           4
4                         Russia           5
..                           ...         ...
225                      Tokelau         226
226  French Southern Territories         227
227                  Saint Lucia         228
228                Cocos Islands         229
229                      Algeria         230

[230 rows x 2 columns]

PK is city:
city       14181
city_id    14181
dtype: int64
                     city  city_id
0                 Xichehe        1
1                  Dolice        2
2                  Wińsko        3
3                Weligama        4
4               Mahalapye        5
...                   ...      ...
14176     Bourg-en-Bresse    14177
14177            Javorník    141

In [490]:
stores = df[[c for c in df.columns if 'store' in c.lower()]].rename(columns={
    'store_name': 'name',
    'store_location': 'location',
    'store_city': 'city',
    'store_state': 'state',
    'store_country': 'country',
    'store_phone': 'phone',
    'store_email': 'email',
})
stores = stores.drop_duplicates()
stores = stores.merge(countries, on='country', how='left').drop(columns='country')
stores = stores.merge(cities, on='city', how='left').drop(columns='city')
stores['store_id'] = stores.index + 1
to_csv(stores, 'stores')
print(f'stores PK is email:\n{stores.nunique()}')
stores

stores PK is email:
name            383
location       4621
state           194
phone         10000
email         10000
country_id      200
city_id        8227
store_id      10000
dtype: int64


,name,location,state,phone,email,country_id,city_id,store_id
0,Youopia,Suite 75,NaN,564-244-8660,bmassingham0@networkadvertising.org,23,1,1
1,Fivespan,Suite 1,NaN,596-788-0321,cscudder1@behance.net,66,2,2
2,Avamba,Suite 69,NaN,148-671-4697,vhuxter2@marketwatch.com,74,3,3
3,Feedfish,Suite 77,NaN,190-352-7524,bbullier3@infoseek.co.jp,56,4,4
4,Skimia,Apt 112,NaN,364-385-6169,sedgar4@e-recht24.de,20,5,5
...,...,...,...,...,...,...,...,...
9995,Wikivu,7th Floor,NaN,404-942-8512,mkardosrn@hp.com,38,3635,9996
9996,Mymm,Apt 1734,NaN,823-500-2552,ikingzettro@tinyurl.com,24,3996,9997
9997,Livefish,PO Box 27778,NaN,430-848-0379,sdrakeleyrp@oakley.com,1,8225,9998
9998,Oodoo,PO Box 7535,NaN,816-976-1481,pemesrq@shop-pro.jp,84,8226,9999


In [491]:
pet_types = df[['customer_pet_type']].drop_duplicates().rename(columns={'customer_pet_type': 'type'}).reset_index().drop(columns='index')
pet_types['pet_type_id'] = pet_types.index + 1
to_csv(pet_types, 'pet_types')

pet_breeds = df[['customer_pet_breed']].drop_duplicates().rename(columns={'customer_pet_breed': 'breed'}).reset_index().drop(columns='index')
pet_breeds['pet_breed_id'] = pet_breeds.index + 1
to_csv(pet_breeds, 'pet_breeds')

pet_categories = df[['pet_category']].drop_duplicates().rename(columns={'pet_category': 'category'}).reset_index().drop(columns='index')
pet_categories['pet_category_id'] = pet_categories.index + 1
to_csv(pet_categories, 'pet_categories')

customers = df[[c for c in df.columns if 'customer' in c.lower() or 'pet' in c.lower()]].rename(columns={
    'sale_customer_id': 'id',
    'customer_first_name': 'first_name',
    'customer_last_name': 'last_name',
    'customer_age': 'age',
    'customer_email': 'email',
    'customer_postal_code': 'postal_code',
    'customer_pet_name': 'pet_name'
})
customers = customers.merge(pet_types, left_on='customer_pet_type', right_on='type', how='left').drop(columns=['customer_pet_type', 'type'])
customers = customers.merge(pet_breeds, left_on='customer_pet_breed', right_on='breed', how='left').drop(columns=['customer_pet_breed', 'breed'])
customers = customers.merge(pet_categories, left_on='pet_category', right_on='category', how='left').drop(columns=['pet_category', 'category'])
customers = customers.merge(countries, left_on='customer_country', right_on='country', how='left').drop(columns=['customer_country', 'country'])
to_csv(customers, 'customers')
print(f'\ncustomers PK is id:\n{customers.nunique()}')
customers


customers PK is id:
first_name          5749
last_name           9053
age                   68
email              10000
postal_code         4036
pet_name            5846
id                 10000
pet_type_id            3
pet_breed_id           3
pet_category_id        5
country_id           204
dtype: int64


,first_name,last_name,age,email,postal_code,pet_name,id,pet_type_id,pet_breed_id,pet_category_id,country_id
0,Barron,Rawlyns,61,bmassingham0@army.mil,NaN,Priscella,1,1,1,1,1
1,Ham,Knowller,78,cscudder1@time.com,73-115,Dalenna,2,2,1,1,2
2,Farleigh,Langley,71,vhuxter2@fotki.com,56-160,Aldridge,3,2,2,2,2
3,Shae,Debling,28,bbullier3@bravesites.com,81700,Beverie,4,1,1,2,3
4,Lay,Twatt,25,sedgar4@smugmug.com,NaN,Sydelle,5,2,3,3,4
...,...,...,...,...,...,...,...,...,...,...,...
9995,Perry,Edwins,49,mkardosrn@jimdo.com,NaN,Warden,9996,1,1,5,24
9996,Benedick,Sherwell,71,ikingzettro@salon.com,NaN,Lezley,9997,2,1,2,55
9997,Lombard,Oxtarby,42,sdrakeleyrp@bluehost.com,NaN,Kassia,9998,2,2,1,24
9998,Yetta,Huertas,26,pemesrq@wikimedia.org,70051,Berri,9999,1,3,1,27


In [492]:
sellers = df[[c for c in df.columns if 'seller' in c.lower()]].rename(columns={
    'sale_seller_id': 'id',
    'seller_first_name': 'first_name',
    'seller_last_name': 'last_name',
    'seller_email': 'email',
    'seller_postal_code': 'postal_code',
})
sellers = sellers.merge(countries, left_on='seller_country', right_on='country', how='left').drop(columns=['seller_country', 'country'])
to_csv(sellers, 'sellers')
print(f'sellers PK is email:\n{sellers.nunique()}')
sellers

sellers PK is email:
first_name      5787
last_name       9025
email          10000
postal_code     3980
id             10000
country_id       206
dtype: int64


,first_name,last_name,email,postal_code,id,country_id
0,Bevan,Massingham,bmassingham0@answers.com,NaN,1,24
1,Candide,Scudder,cscudder1@xing.com,04015,2,67
2,Vinson,Huxter,vhuxter2@issuu.com,NaN,3,1
3,Bald,Bullier,bbullier3@imageshack.us,NaN,4,1
4,Sydelle,Edgar,sedgar4@sogou.com,NaN,5,1
...,...,...,...,...,...,...
9995,Milty,Kardos,mkardosrn@miitbeian.gov.cn,09500-000,9996,13
9996,Isidora,Kingzett,ikingzettro@blogtalkradio.com,NaN,9997,1
9997,Sabine,Drakeley,sdrakeleyrp@addtoany.com,75216,9998,23
9998,Patric,Emes,pemesrq@netvibes.com,8708,9999,20


In [493]:
suppliers = df[[c for c in df.columns if 'supplier' in c.lower()]].rename(columns={
    'supplier_name': 'name',
    'supplier_contact': 'contact',
    'supplier_email': 'email',
    'supplier_phone': 'phone',
    'supplier_address': 'address',
    'supplier_country': 'country',
    'supplier_city': 'city',
})
suppliers = suppliers.merge(countries, on='country', how='left').drop(columns='country')
suppliers = suppliers.merge(cities, on='city', how='left').drop(columns='city')
suppliers['id'] = suppliers.index + 1
to_csv(suppliers, 'suppliers')
print(f'suppliers PK is email:\n{suppliers.nunique()}')
suppliers

suppliers PK is email:
name            383
contact       10000
email         10000
phone         10000
address        4619
country_id      203
city_id        8211
id            10000
dtype: int64


,name,contact,email,phone,address,country_id,city_id,id
0,Tagcat,Bevan Massingham,bmassingham0@unblog.fr,914-877-7062,Suite 25,1,7433,1
1,Livetube,Candide Scudder,cscudder1@sbwire.com,863-319-5653,18th Floor,2,8228,2
2,Photobug,Vinson Huxter,vhuxter2@slate.com,434-817-1275,Apt 96,46,8229,3
3,Janyx,Bald Bullier,bbullier3@dot.gov,515-756-7392,12th Floor,24,8230,4
4,Lazz,Sydelle Edgar,sedgar4@reverbnation.com,861-980-8811,Suite 79,1,8231,5
...,...,...,...,...,...,...,...,...
9995,Quire,Milty Kardos,mkardosrn@barnesandnoble.com,688-242-0789,Suite 47,13,14180,9996
9996,Eimbee,Isidora Kingzett,ikingzettro@eepurl.com,279-253-6631,Apt 79,21,13322,9997
9997,Dablist,Sabine Drakeley,sdrakeleyrp@timesonline.co.uk,469-421-9241,PO Box 84909,8,1383,9998
9998,Katz,Patric Emes,pemesrq@bizjournals.com,855-954-7325,Room 1617,146,1968,9999


In [494]:
products = df[[c for c in df.columns if 'product' in c.lower()]].rename(columns={
    'sale_product_id': 'product_id',
    'product_name': 'name',
    'product_category': 'category',
    'product_price': 'price',
    'product_quantity': 'quantity',
    'product_weight': 'weight',
    'product_color': 'color',
    'product_size': 'size',
    'product_brand': 'brand',
    'product_material': 'material',
    'product_description': 'description',
    'product_rating': 'rating',
    'product_reviews': 'reviews',
    'product_release_date': 'release_date',
    'product_expiry_date': 'expiry_date',
})

product_names = products[['name']].drop_duplicates().reset_index()[['name']]
product_names['product_name_id'] = product_names.index + 1
to_csv(product_names, 'product_names')

product_categories = products[['category']].drop_duplicates().reset_index()[['category']]
product_categories['product_category_id'] = product_categories.index + 1
to_csv(product_categories, 'product_categories')

product_sizes = products[['size']].drop_duplicates().reset_index()[['size']]
product_sizes['product_size_id'] = product_sizes.index + 1
to_csv(product_sizes, 'product_sizes')

product_colors = products[['color']].drop_duplicates().reset_index()[['color']]
product_colors['product_color_id'] = product_colors.index + 1
to_csv(product_colors, 'product_colors')

product_brands = products[['brand']].drop_duplicates().reset_index()[['brand']]
product_brands['product_brand_id'] = product_brands.index + 1
to_csv(product_brands, 'product_brands')

product_materials = products[['material']].drop_duplicates().reset_index()[['material']]
product_materials['product_material_id'] = product_materials.index + 1
to_csv(product_materials, 'product_materials')

products = products.merge(product_names, on='name', how='left').drop(columns=['name'])
products = products.merge(product_categories, on='category', how='left').drop(columns=['category'])
products = products.merge(product_sizes, on='size', how='left').drop(columns=['size'])
products = products.merge(product_colors, on='color', how='left').drop(columns=['color'])
products = products.merge(product_brands, on='brand', how='left').drop(columns=['brand'])
products = products.merge(product_materials, on='material', how='left').drop(columns=['material'])
products['id'] = products.index + 1
products = products.drop_duplicates()

to_csv(products, 'products')
products

,price,quantity,product_id,weight,description,rating,reviews,release_date,expiry_date,product_name_id,product_category_id,product_size_id,product_color_id,product_brand_id,product_material_id,id
0,77.97,89,1,13.4,Aliquam quis turpis eget elit sodales sceleris...,2.1,97,10/19/2011,10/21/2028,1,1,1,1,1,1,1
1,48.70,73,2,36.3,Phasellus in felis. Donec semper sapien a libe...,2.4,37,4/17/2019,2/29/2028,2,1,2,2,2,2,2
2,77.75,39,3,14.6,"Maecenas leo odio, condimentum id, luctus nec,...",3.0,218,2/3/2010,9/3/2023,3,1,1,2,3,2,3
3,54.18,68,4,15.7,Nullam sit amet turpis elementum ligula vehicu...,2.2,959,8/22/2019,3/29/2026,1,1,1,1,4,3,4
4,72.96,23,5,38.9,Quisque id justo sit amet sapien dignissim ves...,4.5,130,12/9/2018,5/10/2030,3,2,3,3,5,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,17.02,53,9996,2.9,Phasellus sit amet erat. Nulla tempus. Vivamus...,2.6,355,1/15/2019,4/9/2024,1,3,1,3,65,2,9996
9996,57.64,81,9997,48.2,"Sed sagittis. Nam congue, risus semper porta v...",1.3,558,5/25/2014,8/31/2025,3,1,3,3,127,4,9997
9997,9.74,46,9998,13.9,Nulla ut erat id mauris vulputate elementum. N...,3.5,55,4/11/2010,3/18/2030,2,3,2,4,137,9,9998
9998,47.26,19,9999,27.6,Phasellus sit amet erat. Nulla tempus. Vivamus...,3.7,269,12/28/2021,7/29/2029,1,3,1,13,269,6,9999


In [495]:
sales = df.loc[:, df.columns.isin(
    [c for c in df.columns if 'sale' in c.lower()] + ['id']
)]

to_csv(sales, 'sales')
sales

,id,sale_date,sale_customer_id,sale_seller_id,sale_product_id,sale_quantity,sale_total_price
0,1,5/14/2021,1,1,1,4,487.70
1,2,11/13/2021,2,2,2,10,484.61
2,3,12/4/2021,3,3,3,9,144.24
3,4,8/10/2021,4,4,4,4,441.09
4,5,2/4/2021,5,5,5,6,72.27
...,...,...,...,...,...,...,...
9995,9996,4/7/2021,9996,9996,9996,7,214.00
9996,9997,2/22/2021,9997,9997,9997,4,495.11
9997,9998,10/15/2021,9998,9998,9998,10,267.09
9998,9999,8/4/2021,9999,9999,9999,4,334.71


# Результат
![image.png](diagram.png)